In [2]:
from IPython.display import clear_output
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

# Data Preprocessing

In [20]:
train = pd.read_csv("/home/phus/Desktop/DS-ISLAND/Kaggle/Data/Spaceship-titanic/train.csv")
test = pd.read_csv("/home/phus/Desktop/DS-ISLAND/Kaggle/Data/Spaceship-titanic/test.csv")
submission = pd.read_csv("/home/phus/Desktop/DS-ISLAND/Kaggle/Data/Spaceship-titanic/sample_submission.csv")

TARGET = 'Transported'

In [21]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
imputer = SimpleImputer(strategy="median" )
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])
train["HomePlanet"].fillna('Z', inplace=True)
test["HomePlanet"].fillna('Z', inplace=True)

In [22]:
label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train ,test = label_encoder(train,test ,label_cols)

In [23]:
train.drop(["Name" ,"Cabin", "PassengerId"] , axis = 1 ,inplace = True)
test.drop(["Name" ,"Cabin", "PassengerId"] , axis = 1 ,inplace = True)
X = train.drop(TARGET , axis =1 )
y = train[TARGET]
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 12 ,
                                                       test_size =0.33)

In [24]:
X_train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
2033,0,1,1,27.0,0,0.0,0.0,0.0,0.0,0.0
1516,0,0,2,42.0,0,0.0,865.0,0.0,0.0,44.0
623,2,1,2,58.0,0,0.0,0.0,0.0,0.0,0.0
6751,1,1,0,22.0,0,0.0,0.0,0.0,0.0,0.0
1648,0,0,2,21.0,0,0.0,0.0,0.0,2.0,807.0
...,...,...,...,...,...,...,...,...,...,...
278,0,0,2,18.0,0,0.0,0.0,1359.0,0.0,0.0
3714,0,0,2,4.0,0,0.0,0.0,0.0,0.0,0.0
7409,2,1,2,24.0,0,0.0,0.0,0.0,0.0,0.0
3325,2,1,2,19.0,0,0.0,0.0,0.0,0.0,0.0


# Modeling

In [25]:
from __future__ import print_function
import numpy as np
from scipy.spatial.distance import cdist
import random

In [53]:
def kmeans_init_centroids (X, k):
    # randomly select k rows of X as initial centroids
    return X[np.random.choice(X.shape[0], k, replace = False)]

def kmeans_assign_labels(X, centroids):
    # calculate pairwise distances btw data and centroids
    D = cdist(X, centroids)
    # return index of the closest centroid
    return np.argmin(D, axis=1)

def has_converged(centroids, new_centroids):
    # return True if two sets of centroids are the same
    return (set([tuple(a) for a in centroids])) == set([tuple(a) for a in new_centroids])

def kmeans_update_centroids(X, labels, K):
    centroids = np.zeros((K,  X.shape[1]))
    
    for k in range(K):
        # collect all points that are assigned to the k-th cluster
        Xk = X[labels == k, :]
        centroids[k, :] = np.mean(Xk, axis=0)   # then take avg
        
    return centroids

def euclidean(point, data):
    """
    Euclidean distance between point & data.
    Point has dimensions (m,), data has dimensions (n,m), and output will be of size (n,).
    """
    return np.sqrt(np.sum((point - data)**2, axis=1))

def evaluate_kmeans(X, K, centroids):
    centroids_found = []
    centroid_idxs = []
    for x in X:
        dists = euclidean(x, centroids)
        centroid_idx = np.argmin(dists)
        centroids_found.append(centroids[centroid_idx])
        centroid_idxs.append(centroid_idx)
    return centroids_found, centroid_idxs

def kmeans(X, K):
    centroids = [kmeans_init_centroids(X, K)]
    labels = []
    it = 0
    while True:
        labels.append(kmeans_assign_labels(X, centroids[-1]))
        new_centroids = kmeans_update_centroids(X, labels[-1], K)
        if has_converged(centroids[-1], new_centroids):
            break
        centroids.append(new_centroids)
        it+=1
    return (centroids, labels, it)

In [42]:
# transform data to two dimensions using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X)
pca_2d = pca.transform(X)
pca_2d

array([[ -577.77373452,  -216.15425634],
       [ -407.10799042,   148.50676695],
       [ 4567.21771111,  2540.15362518],
       ...,
       [ -585.97255031,  -199.13171493],
       [ 1397.92104075,  1796.25369392],
       [ 3732.74108539, -2054.89443566]])

In [68]:
(centroids, labels, it) = kmeans(np.array(X), 2)

print('Centers found by our algorithm:\n', centroids[-1]) 
print('Labels found by our algorithm:\n', labels[-1])

Centers found by our algorithm:
 [[7.06094050e-01 4.14467370e-01 1.53310940e+00 2.84695298e+01
  6.65786948e-02 2.22455974e+02 1.92621401e+02 1.71731166e+02
  2.30309741e+02 2.23086612e+02]
 [1.02521008e+00 4.48179272e-02 1.09803922e+00 3.62801120e+01
  1.40056022e-01 1.62879552e+02 6.42169468e+03 1.19162465e+02
  2.03901681e+03 2.05361345e+03]]
Labels found by our algorithm:
 [0 0 1 ... 0 0 1]


Evaluate

In [64]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X)
pca_2d = pca.transform(test)
pca_2d

array([[-577.78961205, -216.16575311],
       [ 206.22610789, 1517.28021446],
       [-577.78420084, -216.16189749],
       ...,
       [-577.78942511, -216.16568591],
       [2032.86623965, -915.06441779],
       [-577.7685223 , -216.15051454]])

In [75]:
centroids_found, centroid_idx = evaluate_kmeans(np.array(test), 2, centroids[-1])

In [76]:
submission[TARGET] = centroid_idx

# if value in Transported column is 0, then change it to False, vice versa
submission[TARGET] = submission[TARGET].apply(lambda x: True if x == 0 else False)

In [77]:
submission.to_csv("submission_tosubmit.csv", index=False)

In [78]:
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,True
2,0019_01,True
3,0021_01,False
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [63]:
len(test)

4277